<h2><b style="color:#30A2FF">AutoComplete code using N-Gram Language Model</b></h2>
1.  N-Gram Language Model is probablistic model witch uses only previous n-1 words for predicting the next word.<br>
2. This estimation is known as <b style="color:#3330E4">Morkov assumption</b><br>
3. This code is divided into four parts-<br>
<em style="margin-left:25px";>a. Data Preprocessing</em><br>
<em style="margin-left:25px";>b. Building N-gram Model</em><br>
<em style="margin-left:25px";>c. Perplexity function</em><br>
<em style="margin-left:25px";>d. Building Auto Complete System</em><br>


In [43]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')

<h3><b style="color:#2146C7;">Part-1 Preprocessing of Data</b></h3>
In this part ,I done preprocessing on <b style="color:#30A2FF;">Twitter data</b> by preforming -<br>
<em style="margin-left:25px";>a. Loading the data</em><br>
<em style="margin-left:25px";>b. Spilting data into senteces</em><br>
<em style="margin-left:25px";>c. Generating the sentence wise tokens using nltk.word_tokenize </em><br>
<em style="margin-left:25px";>d. Train-Test split</em><br>
<em style="margin-left:25px;">e. Creating <b style="color:#30A2FF;">Vocabulary</b> from training data </em><br>

In [2]:
with open("en_US.twitter.txt","r") as f:
    data=f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------


In [3]:
def split_to_sentences(data):
    sentences=data.split('\n')
    sentences=[s.strip() for s in sentences]
    sentences=[s for s in sentences if len(s)>0]
    return sentences

In [4]:
x = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""
print(x)
split_to_sentences(x)



I have a pen.
I have an apple. 
Ah
Apple pen.




['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

In [5]:
'''This Function generates the tokens using nltk.word_tokenize'''
def tokenize_sentences(sentences):
    tokenized_sentences=[]
    for sentence in sentences:
        sentence=sentence.lower()
        tockens=nltk.word_tokenize(sentence)
        tokenized_sentences.append(tockens)
    return tokenized_sentences

In [6]:
'''# test your code'''
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)


[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

In [7]:
def get_tokenized_data(data):
    sentences=split_to_sentences(data)
    return tokenize_sentences(sentences)

In [8]:
'''# test previous function'''
x = "Sky is blue.\nLeaves are green\nRoses are red."
get_tokenized_data(x)


[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

<h4>Train Test split</h4>

In [9]:
'''#Train Test split'''
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]


In [10]:
print("{} data are split into {} train and {} test set".format(
    len(tokenized_data), len(train_data), len(test_data)))

print("First training sample:")
print(train_data[0])

print("First test sample")
print(test_data[0])


47961 data are split into 38368 train and 9593 test set
First training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']
First test sample
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']


<h5>Creating Vocabulary of words having frequency greater than threshold and <br>replacing words which are out of vocabulary by token special <em style="color:#30A2FF;">'unk'</em> </h5>

In [11]:
def count_words(tokenized_sentences):
    word_count={}
    for tokenized_sentence in tokenized_sentences:
        for token in tokenized_sentence:
            if token not in word_count:
                word_count[token] = 0
            word_count[token] += 1
    return word_count


In [12]:
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)


{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}

In [13]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    closed_vocab = []
    word_counts=count_words(tokenized_sentences)
    for word,count in word_counts.items():
        if(count>=count_threshold):
            closed_vocab.append(word)
    return closed_vocab

In [14]:
# test your code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
tmp_closed_vocab = get_words_with_nplus_frequency(
    tokenized_sentences, count_threshold=2)
print(f"Closed vocabulary:")
print(tmp_closed_vocab)


Closed vocabulary:
['.', 'are']


In [15]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []

    for sentence in tokenized_sentences:
        rep_sentence=[]
        for token in sentence:
            if token not in vocabulary:
                rep_sentence.append(unknown_token)
            else:
                rep_sentence.append(token)
        replaced_tokenized_sentences.append(rep_sentence)
    return replaced_tokenized_sentences

In [16]:
tokenized_sentences = [["dogs", "run"], ["cats", "sleep"]]
vocabulary = ["dogs", "sleep"]
tmp_replaced_tokenized_sentences = replace_oov_words_by_unk(
    tokenized_sentences, vocabulary)
print(f"Original sentence:")
print(tokenized_sentences)
print(f"tokenized_sentences with less frequent words converted to '<unk>':")
print(tmp_replaced_tokenized_sentences)


Original sentence:
[['dogs', 'run'], ['cats', 'sleep']]
tokenized_sentences with less frequent words converted to '<unk>':
[['dogs', '<unk>'], ['<unk>', 'sleep']]


In [17]:
def preprocess_data(train_data, test_data, count_threshold):
    vocabulary=get_words_with_nplus_frequency(train_data,count_threshold)
    train_data_replaced=replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced=replace_oov_words_by_unk(test_data, vocabulary)
    return train_data_replaced, test_data_replaced,vocabulary


In [18]:
'''# test previous code'''
tmp_train = [['sky', 'is', 'blue', '.'],
             ['leaves', 'are', 'green']]
tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_data(tmp_train,tmp_test,count_threshold=1)
print("tmp_train_repl")
print(tmp_train_repl)
print()
print("tmp_test_repl")
print(tmp_test_repl)
print()
print("tmp_vocab")
print(tmp_vocab)


tmp_train_repl
[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]

tmp_test_repl
[['<unk>', 'are', '<unk>', '.']]

tmp_vocab
['sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


In [19]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data,test_data, minimum_freq)

In [20]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))


First preprocessed training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']

First preprocessed test sample:
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']

First 10 vocabulary:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the']

Size of vocabulary: 14823


<h3><b style="color:#2146C7;">Part-2 Buiding N_Gram Model</b></h3>
In this part ,I built functions for buiding N-gram Model -<br>
<em style="margin-left:25px";>a. Count_n_gram -for generating desired N_gram with frequnecy</em><br>
<em style="margin-left:25px";>b. <b style="color:#30A2FF;">Estimate_probabilty</b> - Probability of each word based on previos words,n_gram and n+1_gram</em><br>
<em style="margin-left:25px";>c. Genarating Count matrix</em><br>


In [21]:
'''This function gernarates desired N_gram and also addas start and end tokens'''
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    n_grams={}
    for sentence in data:
        sentence=[start_token]*n+sentence+[end_token]
        for i in range(0,len(sentence)-n+1):
            n_gram=tuple(sentence[i:i+n])
            if n_gram  not in n_grams:
                n_grams[n_gram]=0
            n_grams[n_gram]+=1
    return n_grams


In [22]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))


Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


estimated_P=C(<em>w<sub>t-1</sub></em>....<em>w<sub>t-n</sub></em>,<em>w<sub>n</sub></em>) / C(<em>w<sub>t-1</sub></em>....<em>w<sub>t-n</sub></em>)

<b>Add - K smoothing - </b><br>
To avoid Zero probalbility case I used Add -k smoothing Therefore,<br>
<b style="color:#30A2FF"><em>estimated_P</em>=C(<em>w<sub>t-1</sub></em>....<em>w<sub>t-n</sub></em>,<em>w<sub>n</sub></em>) +k  / C(<em>w<sub>t-1</sub></em>....<em>w<sub>t-n</sub></em>+|v|*k)</b>


In [23]:
'''#Probability estimation of word'''
def estimate_probability(word,previous_n_gram,n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    previous_n_gram = tuple(previous_n_gram)
    count_previous=n_gram_counts.get(previous_n_gram,0)
    current_n_gram=previous_n_gram+(word,)
    count_current=n_plus1_gram_counts.get(current_n_gram,0)
    prob = (count_current+k)/(count_previous+(vocabulary_size*k))
    probability=prob
    return probability


In [24]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)

bigram_counts = count_n_grams(sentences, 2)

tmp_prob = estimate_probability(
    "cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

print(
    f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob}")



The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333333333333333


In [25]:
'''Generating a dictiinary of word'''
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    previous_n_gram=tuple(previous_n_gram)
    vocabulary_size = len(vocabulary)
    vocabulary = vocabulary + ["<e>", "<unk>"]
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(
            word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=k)
        probabilities[word]=probability
    return probabilities

In [26]:
'''# test previous  code'''
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)


{'i': 0.1111111111111111,
 'dog': 0.1111111111111111,
 'like': 0.1111111111111111,
 'cat': 0.3333333333333333,
 'this': 0.1111111111111111,
 'is': 0.1111111111111111,
 'a': 0.1111111111111111,
 '<e>': 0.1111111111111111,
 '<unk>': 0.1111111111111111}

In [27]:
'''# Additional test'''
trigram_counts = count_n_grams(sentences, 3)
estimate_probabilities(["<s>", "<s>"], bigram_counts,trigram_counts, unique_words, k=1)


{'i': 0.2222222222222222,
 'dog': 0.1111111111111111,
 'like': 0.1111111111111111,
 'cat': 0.1111111111111111,
 'this': 0.2222222222222222,
 'is': 0.1111111111111111,
 'a': 0.1111111111111111,
 '<e>': 0.1111111111111111,
 '<unk>': 0.1111111111111111}

In [28]:
'''Generating Count _matrix fo N_Gram model'''
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    vocabulary = vocabulary + ["<e>", "<unk>"]
    n_grams=[]
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram=n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams=list(set(n_grams))
    nrow = len(n_grams)
    ncol = len(vocabulary)
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    # mapping from next word to column
    col_index = {word:j for j, word in enumerate(vocabulary)}
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count

    count_matrix = pd.DataFrame(
        count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix


In [29]:
'''#test of previous code'''
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)

print('bigram counts')
display(make_count_matrix(bigram_counts, unique_words))


bigram counts


,i,dog,like,cat,this,is,a,<e>,<unk>
"(i,)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(this,)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>,)",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(dog,)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(is,)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(like,)",0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
"(a,)",0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


In [30]:
'''# Show trigram counts'''
print('\ntrigram counts')
trigram_counts = count_n_grams(sentences, 3)
display(make_count_matrix(trigram_counts, unique_words))



trigram counts


,i,dog,like,cat,this,is,a,<e>,<unk>
"(is, like)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(this, dog)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(<s>, i)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(i, like)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(like, a)",0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
"(a, cat)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(<s>, <s>)",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(<s>, this)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(dog, is)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3><b style="color:#2146C7;">Part-3 Perplexity</b></h3>
Perplexity is evaluation metric for language Model.
To calculate Perplexity score of the test set on n_gram model,<br>
<em style="color:#30A2FF;">log<sub>2</sub> (Perplexity) = (-1 \ N) log<sub>2</sub> <b> &sum;<sub>t=n</sub><sup>N-1</sup> P( <em>w<sub>t</sub> | <em>w<sub>t-1</sub></em>....<em>w<sub>t-n</sub></em>)</b></em></em><br>
where <br>
<em style="margin-left:25px";><b>N</b> is the length of the sentence.</em><br>
<em style="margin-left:25px";><b>n</b> is the number of words in the n-gram (e.g. 2 for a bigram)..</em><br><br>
The higher the probabilities are, the lower the perplexity will be.<br>
The more the n-grams tell us about the sentence, the lower the perplexity score will be.

In [31]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    n = len(list(n_gram_counts.keys())[0]) 
    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)
    N = len(sentence)
    product_pi = 1.0
    for i in range(n,N):
        previous_n_gram=list(sentence[i-n:i])
        word = sentence[i]
        prob = estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=k)
        product_pi*=1/prob
        perplexity = product_pi**(1/float(N))
    return perplexity

In [32]:
'''#test previous code'''
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train1 = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexity for first train sample: {perplexity_train1:.4f}")

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print(f"Perplexity for test sample: {perplexity_test:.4f}")


Perplexity for first train sample: 2.8040
Perplexity for test sample: 3.9654


<h3><b style="color:#2146C7;">Part-4 Buiding AutoComplete System</b></h3>
In this part ,I built functions for buiding AutoComplete System - <br>
<em style="margin-left:25px";>a. suggest_a_word -for suggesting best possible word using N-gram model</em><br>
<em style="margin-left:25px";>b.Testing </me>


In [33]:
'''suggesting best word from previous tokens'''
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    # length of previous words
    n = len(list(n_gram_counts.keys())[0])
    # From the words that the user already typed
    # get the most recent 'n' words as the previous n-gram
    previous_n_gram = previous_tokens[-n:]
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    suggestion = None
    max_prob = 0

    for word,prob in probabilities.items():
        if(start_with!=None):
            if(not word.startswith(start_with)):
                continue
        if max_prob < prob:
            suggestion = word
            max_prob = prob

    return (suggestion,max_prob)


In [34]:
'''#test of previous code'''
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(
    previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(
    f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts,
                              bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(
    f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")


The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.3333

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.1111


In [35]:
'''Get Suggestion function gives the suggestion list from all n_granms in n_gram_counts_list'''
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]

        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions


In [36]:
'''# test previous code'''
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts,
                      trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
tmp_suggest3 = get_suggestions(
    previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"The previous words are 'i like', the suggestions are:")
display(tmp_suggest3)


The previous words are 'i like', the suggestions are:


[('a', 0.3333333333333333),
 ('a', 0.25),
 ('i', 0.14285714285714285),
 ('i', 0.14285714285714285)]

<h4>Using<b style="color:#30A2FF">Training Data </b>for gernerating suggestions</h4>

In [37]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)


Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [38]:
previous_tokens = ["i", "am", "to"]
tmp_suggest4 = get_suggestions(
    previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)


The previous words are ['i', 'am', 'to'], the suggestions are:


[('be', 0.027665685098338604),
 ('have', 0.00013487086115044844),
 ('have', 0.00013490725126475548),
 ('i', 6.746272684341901e-05)]

In [39]:
previous_tokens = ["i", "want", "to", "go"]
tmp_suggest5 = get_suggestions(
    previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest5)


The previous words are ['i', 'want', 'to', 'go'], the suggestions are:


[('to', 0.014051961029228078),
 ('to', 0.004697942168993581),
 ('to', 0.0009424436216762033),
 ('to', 0.0004044489383215369)]

In [40]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest7 = get_suggestions(
    previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest7)


The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[("'re", 0.023973994311255586),
 ('?', 0.002888465830762161),
 ('?', 0.0016134453781512605),
 ('<e>', 0.00013491635186184566)]

In [41]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest8 = get_suggestions(
    previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="d")

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest8)


The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[('do', 0.009020723283218204),
 ('doing', 0.0016411737674785006),
 ('doing', 0.00047058823529411766),
 ('dvd', 6.745817593092283e-05)]

<h6>1.Refernce: deeplearning.ai NLP specialization</h6>